In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import logging
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

In [2]:
# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("scraper.log"),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

In [3]:
def extract_table_data(table):
    """Extracts data from an HTML table into a list of rows."""
    data = []
    for row in table.find_all('tr'):
        # Get both header and data cells
        cells = row.find_all(['td', 'th'])
        # Clean up the text from each cell
        row_data = [cell.get_text(strip=True) for cell in cells if cell.get_text(strip=True)]
        if row_data:
            data.append(row_data)
    return data

In [4]:
# --------------------------
# NECTA Results Page Scraping
# --------------------------

def scrape_necta_performance(url):
    """Scrapes the given URL and returns the two performance sections' data."""
    response = requests.get(url)
    if response.status_code != 200:
        return ['TOTAL PASSED CANDIDATES',None],['GPA',None],None,None
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find the overall performance section heading and then its table
    overall_heading = soup.find(lambda tag: tag.name in ["h1", "h2", "td", "th"] and "EXAMINATION CENTRE OVERALL PERFORMANCE" in tag.get_text())
    overall_data = None
    if overall_heading:
        overall_table = overall_heading.find_next('table')
        if overall_table:
            overall_data = extract_table_data(overall_table)
    
    # Find the division performance section heading and then its table
    division_heading = soup.find(lambda tag: tag.name in ["h1", "h2", "td", "th"] and "EXAMINATION CENTRE DIVISION PERFORMANCE" in tag.get_text())
    division_data = None
    if division_heading:
        division_table = division_heading.find_next('table')
        if division_table:
            division_data = extract_table_data(division_table)
    #passed student list,gpa list, registered,sat for exam
    return overall_data[0],overall_data[1], division_data[0][0],division_data[0][2]
    

In [5]:
def scrape_school_features(url):
    """
    Scrapes the school profile page to extract:
    1. Feature availability from the container with class 'listed_features'
    
    Returns a single dictionary with all features and counts.
    """
    features = {}
    
    try:
        response = requests.get(url, timeout=10)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Extract features
        features_container = soup.find('div', class_='listed_features')
        if features_container:
            feature_divs = features_container.find_all('div', recursive=False)
            for div in feature_divs:
                span = div.find('span')
                feature_name = span.get_text(strip=True) if span else None
                icon = div.find('i')
                # Check for the icon class: checkmark means available
                status = "Available" if icon and "icon-checkmark" in icon.get("class", []) else "Not Available"
                if feature_name:
                    features[feature_name.upper()] = status
        
        
        return features
    except Exception as e:
        print("Error scraping school data:", e)
        return features

In [6]:
def parse_school_profile_features(features):
    """
    Interprets the features dictionary to produce:
      - SCHOOL OWNERSHIP (GOVERNMENT or PRIVATE)
      - SCHOOL CATEGORY (BOYS ONLY, GIRLS ONLY, or BOYS AND GIRLS)
      - SCHOOL TYPE (DAY, BOARDING, or DAY AND BOARDING)
      - ACADEMIC LEVEL CATEGORY (ALEVEL ONLY or COMBINED OA)
    """
    # Determine SCHOOL OWNERSHIP
    if features.get("GOVERNMENT", "Not Available") == "Available":
        school_ownership = "GOVERNMENT"
    elif features.get("PRIVATE", "Not Available") == "Available":
        school_ownership = "PRIVATE"
    else:
        school_ownership = None
    
    # Determine SCHOOL CATEGORY
    if features.get("BOYS ONLY", "Not Available") == "Available":
        school_category = "BOYS ONLY"
    elif features.get("GIRLS ONLY", "Not Available") == "Available":
        school_category = "GIRLS ONLY"
    elif features.get("BOYS & GIRLS", "Not Available") == "Available" or features.get("BOYS AND GIRLS", "Not Available") == "Available":
        school_category = "BOYS AND GIRLS"
    else:
        school_category = None
    
    # Determine SCHOOL TYPE
    day_status = features.get("DAY", "Not Available")
    boarding_status = features.get("BOARDING", "Not Available")
    if day_status == "Available" and boarding_status == "Available":
        school_type = "DAY AND BOARDING"
    elif day_status == "Available":
        school_type = "DAY"
    elif boarding_status == "Available":
        school_type = "BOARDING"
    else:
        school_type = None
    
    # Determine ACADEMIC LEVEL CATEGORY
    # Check for both O-LEVEL and A-LEVEL availability.
    o_level = features.get("O-LEVEL", features.get("OLEVEL", "Not Available"))
    a_level = features.get("A-LEVEL", features.get("ALEVEL", "Not Available"))
    if a_level == "Available" and o_level == "Available":
        academic_level_category = "COMBINED OA"
    elif a_level == "Available":
        academic_level_category = "ALEVEL ONLY"
    else:
        academic_level_category = None  
    
    return school_ownership, school_category, school_type, academic_level_category

In [7]:
def parse_counts(counts):
      #Determine number.
    students = counts.get("STUDENTS","Not Available")
    teachers = counts.get("TEACHERS","Not Available")
    students_teacher_ratio = counts.get("STUDENT-TEACHER RATIO","Not Available")

    return students,teachers,students_teacher_ratio

In [8]:

# --------------------------
# Academics Page Scraping
# --------------------------
def scrape_advanced_level_combinations(url):
    """
    Scrapes the academics page for Advanced Level Combinations.
    Looks for a <div class="sub-comb"> where the header contains "Advanced Level Combinations",
    then collects all combination texts from the <p> tags in the sibling div.
        2. Staff and student information from the container with class 'count-staff-students'

    """
    try:
        response = requests.get(url, timeout=10)
        soup = BeautifulSoup(response.content, 'html.parser')
        sub_comb_divs = soup.find_all('div', class_='sub-comb')
        combinations = []
        for sub_comb in sub_comb_divs:
            header_div = sub_comb.find('div', class_='profile_facility_symbol')
            if header_div and "Advanced Level Combinations" in header_div.get_text():
                combination_container = header_div.find_next_sibling('div')
                if combination_container:
                    p_tags = combination_container.find_all('p')
                    for p in p_tags:
                        comb_text = p.get_text(strip=True)
                        if comb_text:
                            combinations.append(comb_text)
                break


        counts_container = soup.find('div', class_='count-staff-students')
        counts = {}
        if counts_container:
            count_divs = counts_container.find_all('div', recursive=False)
            for div in count_divs:
                # Find the text before the colon
                text_content = div.get_text(strip=True)
                if ':' in text_content:
                    label, _ = text_content.split(':', 1)
                    # Clean up the label
                    label = label.replace('Number of', '').strip()
                    # Extract the value from the span
                    span_value = div.find('span')
                    if span_value:
                        clean_value = span_value.get_text(strip=True)
                        counts[label.upper()] = clean_value
                        
        return combinations, counts
    except Exception as e:
        print("Error scraping advanced level combinations:", e)
        return []



In [9]:
def get_combination_category(combination_list):
    """
    Determines the overall combinations category based on the list of combination abbreviations.
    Uses a mapping (e.g., 'PCM' and 'PCB' map to SCIENCE, 'HGL' maps to ARTS).
    Returns a single category or 'MIXED' if multiple categories are found.
    """
    category_mapping = {
        'PCM': 'SCIENCE',
        'PCB': 'SCIENCE',
        'CBG' : 'SCIENCE',
        'PGM' : 'SCIENCE',
        'CBM' : 'SCIENCE',
        'HGL': 'ARTS',
        'HKL' : 'ARTS',
        'HGK' : 'ARTS',
        'HGE' : 'ARTS',
        'ECA' : 'BUSSINESS',
        'ECG' : 'BUSSINESS',
        'EGM' : 'BUSSINESS',
        'HEA' : 'BUSSINESS',
        'ECM' : 'BUSSINESS',
        'PEM' : 'MIXED',
        'GHE' : 'MIXED',
        'PCE' : 'MIXED',
        'CBE' : 'MIXED',
        'MEC' : 'MIXED',
    }
    categories_found = set()
    for comb in combination_list:
        normalized = comb.strip().upper()
        if normalized in category_mapping:
            categories_found.add(category_mapping[normalized])
        else:
            categories_found.add("UNKNOWN")
    if len(categories_found) == 1:
        return categories_found.pop()
    else:
        return "MIXED"


In [10]:
def extract(year=2024,school_no=101):
    # Define URLs
    # necta_url = "https://onlinesys.necta.go.tz/results/2023/acsee/results/s0110.htm"
    # profile_url = "https://www.school.co.tz/s0110/school-profile"
    # academics_url = "https://www.school.co.tz/s0110/academics"

    necta_url = f"https://onlinesys.necta.go.tz/results/2023/acsee/results/s{school_no:04d}.htm"
    profile_url = f"https://www.school.co.tz/s{school_no:04d}/school-profile"
    academics_url = f"https://www.school.co.tz/s{school_no:04d}/academics"
    
    # 1. Scrape NUMBER OF CANDIDATES from NECTA results page
    students,gpa,registered,sat = scrape_necta_performance(necta_url)
    
    # 2. Scrape school features from the profile page and interpret them
    features = scrape_school_features(profile_url)
    school_ownership, school_category, school_type, academic_level_category = parse_school_profile_features(features)
    
    # 3. Scrape Advanced Level Combinations from the academics page and determine its category
    combinations,counts = scrape_advanced_level_combinations(academics_url)
    students_count,teachers_count,ratio = parse_counts(counts)
    combinations_category = get_combination_category(combinations) if combinations else None
    
    # Combine all extracted data into a single dictionary
    data = {
        "SCHOOL" : f"s{school_no:04d}",
        "NUMBER OF CANDIDATES": registered,
        "SAT FOR EXAM" : sat,
        "SCHOOL OWNERSHIP": school_ownership,
        "SCHOOL CATEGORY": school_category,
        "SCHOOL TYPE": school_type,
        "COMBINATIONS CATEGORY": combinations_category,
        "ACADEMIC LEVEL CATEGORY": academic_level_category,
        students[0] : students[1],
        "STUDENTS" : students_count,
        "TEACHERS" : teachers_count,
        "STUDENT-TEACHER RATIO" : ratio,
        "GPA" : gpa[1][:6] if gpa[1] else None
    }
    
    # Create a DataFrame with a single row for this school
    df = pd.DataFrame([data])
    return df

In [17]:
school_range = list(range(100, 6278))

# Use ThreadPoolExecutor for parallel processing
with ThreadPoolExecutor(max_workers=4) as executor:
    # Map the extract function across all schools
    futures = list(tqdm(
        executor.map(lambda x: extract(2024, x).set_index("SCHOOL"), school_range),
        total=len(school_range),
        desc="Scraping Schools"
    ))

# Combine all the dataframes
df = pd.concat(futures)
df

Scraping Schools:  12%|█████████▊                                                                          | 718/6178 [11:32<1:03:27,  1.43it/s]

Error scraping advanced level combinations: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))


Scraping Schools:  12%|█████████▊                                                                          | 721/6178 [11:36<1:27:50,  1.04it/s]


ValueError: not enough values to unpack (expected 2, got 0)

In [12]:
df.dropna(axis=0)

,NUMBER OF CANDIDATES,SAT FOR EXAM,SCHOOL OWNERSHIP,SCHOOL CATEGORY,SCHOOL TYPE,COMBINATIONS CATEGORY,ACADEMIC LEVEL CATEGORY,TOTAL PASSED CANDIDATES,STUDENTS,TEACHERS,STUDENT-TEACHER RATIO,GPA
SCHOOL,,,,,,,,,,,,
s0101,197,196,GOVERNMENT,BOYS ONLY,DAY AND BOARDING,MIXED,COMBINED OA,196,1515,80,19:1,2.7577
s0108,282,280,GOVERNMENT,BOYS AND GIRLS,BOARDING,SCIENCE,COMBINED OA,280,1000,80,13:1,2.7375
s0110,184,183,GOVERNMENT,BOYS ONLY,BOARDING,MIXED,COMBINED OA,183,730,60,12:1,1.6548
s0113,22,22,PRIVATE,BOYS ONLY,BOARDING,MIXED,COMBINED OA,22,300,30,10:1,2.7576
s0119,183,183,GOVERNMENT,BOYS ONLY,BOARDING,MIXED,COMBINED OA,183,820,70,12:1,1.6384
s0120,463,463,GOVERNMENT,BOYS ONLY,BOARDING,MIXED,ALEVEL ONLY,463,900,30,30:1,2.4669
s0130,36,36,PRIVATE,BOYS ONLY,BOARDING,MIXED,COMBINED OA,36,300,30,10:1,2.2593
s0133,342,342,GOVERNMENT,BOYS ONLY,BOARDING,MIXED,ALEVEL ONLY,341,800,40,20:1,2.5699
s0140,135,133,GOVERNMENT,BOYS ONLY,BOARDING,MIXED,COMBINED OA,133,660,50,13:1,1.4749


In [13]:
len(df.dropna(axis=0))

46

In [14]:
df.to_csv('(2024).csv')

In [15]:
df.dropna(axis=0).to_csv('school_performance_data(2024)_.csv')